In [1]:
import numpy as np
from math import factorial, comb
from numpy import polynomial as Polynomial
import matplotlib.pyplot as plt
import time
import random
from polynomial_utils import compose, compose_layers, l2_norm, l2_coefficient_norm, plot_polynomials, carleman, carleman_matrix, l_inf_coefficient_norm
from tqdm import tqdm
from scipy.linalg import lstsq

In [2]:
DTYPE = np.float64

In [3]:
def solve_for_h(g, target_poly):

    target_deg = target_poly.degree()
    g_deg = g.degree()
    h_deg = target_deg / g_deg
    if h_deg != int(h_deg):
        raise ValueError("Degree of target poly should be a multiple of g degree")
    h_deg = int(h_deg)

    h = Polynomial.Polynomial(np.array([0] * (h_deg + 1), dtype=DTYPE))
    # For some fucking reason numpy can't take cube roots of negative numbers properly.
    tmp = target_poly.coef[-1]/g.coef[-1]
    is_neg = tmp < 0
    h.coef[-1] = (abs(tmp))**(1/h_deg) * (-1 if is_neg else 1)
    for it, id in enumerate(range(h_deg-1, 0, -1)):
        # There is an affine relationship between the h we're looking for and
        # the coefs we have. To compute the h_coef we're looking for, we need to
        # isolate which part is linear in h_coef and which isn't. Setting
        # h_coef = 0 gives us the offset and h_coef = 1 the total of offset + linear part.
        h_pow1 = h**g_deg   
        h.coef[id] = 1      
        h_pow2 = h**g_deg

        h.coef[id] = (target_poly.coef[-it - 2] - g.coef[-1]*h_pow1.coef[-it - 2]) / (g.coef[-1] * (h_pow2.coef[-it - 2] - h_pow1.coef[-it - 2]))
    
    # h0 has a slightly different formula
    h_pow1 = h**g_deg
    h_pow1 = h**g_deg   
    h.coef[0] = 1
    h_pow2 = h**g_deg
    h.coef[0] = (target_poly.coef[-it - 3] - g.coef[-1]*h_pow1.coef[-it - 3] - g.coef[-2]*h.coef[-1]**(g_deg-1)) / (g.coef[-1] * (h_pow2.coef[-it - 3] - h_pow1.coef[-it - 3]))

    return h

In [4]:
def new_poly(width, n1 = 4, n2 = 4):
    g = Polynomial.Polynomial(np.random.uniform(-width, width, n1))
    h = Polynomial.Polynomial(np.random.uniform(-width, width, n2))
    h.coef = h.coef.astype(DTYPE)
    g.coef = g.coef.astype(DTYPE)
    target_poly = compose(g, h, dtype=DTYPE)
    return g, h, target_poly

In [136]:
g, h, target_poly = new_poly(1, 20, 20)
# print(f"{g} (g)")
# print(f"{h} (true h)")
h_bar = solve_for_h(g, target_poly)
target_bar = compose(g, h_bar, dtype=DTYPE)
# print(f"{h_bar} (computed h)")
print(max(abs(h.coef - h_bar.coef)))
print(max(abs(target_poly.coef - target_bar.coef)))

# print(target_bar)
# print(target_poly)

print(abs(h.coef - h_bar.coef))

6.825973074430641e-05
11553473.620471954
[6.82597307e-05 3.86511842e-05 1.84954418e-05 1.11523827e-06
 1.41315881e-06 4.56090773e-07 1.04001408e-08 2.58094979e-08
 6.07359574e-09 8.56625881e-11 1.82079352e-10 2.02027284e-11
 6.29746255e-12 2.86770607e-12 6.09512441e-13 6.66133815e-14
 2.22044605e-16 6.66133815e-16 0.00000000e+00 0.00000000e+00]


In [138]:
print(h_bar.coef[0])
for j in range(10):
    coef_pows = np.array([h_bar.coef[0] ** i for i in range(g.degree()+1)])
    fxn = g.coef @ coef_pows - target_poly.coef[0]
    deriv_coef_pows = np.array([i * h_bar.coef[0] ** (i-1) for i in range(g.degree()+1)])
    fxn_prime = g.coef @ deriv_coef_pows
    h_bar.coef[0] -= fxn / fxn_prime

print(h_bar.coef[0] - fxn/fxn_prime)
print(h.coef[0])
print(abs(h_bar.coef[0] - h_bar.coef[0] - fxn/fxn_prime))

0.038168829872328065
0.038237089603072566
0.03823708960307237
0.0


In [139]:
target_bar = compose(g, h_bar, dtype=DTYPE)
print(target_poly.coef - target_bar.coef)
print(g.coef[-1] * h_bar.coef[-1]**19 - target_poly.coef[-1])

[ 1.11022302e-16  2.40801680e-05 -4.96786129e-06 -4.85268523e-06
  5.15583919e-05  5.37117883e-05  1.34712623e-05 -2.37060988e-05
  1.86916236e-05  1.63199082e-05 -2.89176489e-05 -8.71741787e-05
 -4.83985647e-05  2.54046342e-04  2.04103369e-05 -6.40704702e-04
 -2.10847649e-05  1.05734739e-03 -3.38557836e-05 -1.28262248e-03
  6.47609327e-04  2.17843381e-03 -7.84296236e-04 -2.36987906e-03
  5.66368939e-04  2.57975288e-03  4.04334119e-04 -5.99973530e-03
 -7.51225280e-03  8.80420528e-03  1.76529947e-02 -1.87241701e-02
 -3.44874381e-02  4.13201107e-02  5.84566173e-02 -8.49327854e-02
 -9.06587251e-02  1.68561149e-01  1.46692289e-01 -3.01211248e-01
 -2.21530473e-01  5.14163886e-01  3.10159506e-01 -8.73598963e-01
 -4.40220611e-01  1.45739105e+00  6.17297824e-01 -2.41310025e+00
 -8.52356326e-01  3.98324333e+00  1.19235883e+00 -6.47942675e+00
 -1.46555187e+00  1.07007733e+01  1.63572861e+00 -1.75394517e+01
 -1.26387603e+00  2.82921713e+01 -7.99849861e-01 -4.51363610e+01
  6.08734334e+00  7.01250

In [7]:
errors = []
for i in range(1000):
    g, h, target_poly = new_poly(1, 10, 10)
    # print(f"{g} (g)")
    # print(f"{h} (true h)")
    h_bar = solve_for_h(g, target_poly)
    target_bar = compose(g, h_bar, dtype=DTYPE)
    # print(f"{h_bar} (computed h)")
    # err = max(abs(h.coef - h_bar.coef))
    err = max(abs(target_poly.coef - target_bar.coef))
    # print(err)
    if np.isinf(err) or np.isnan(err):
        continue
    errors.append(err)

print(f"Max: {max(errors)}")
print(f"Avg: {sum(errors) / len(errors)}")
print(f"Std: {np.std(errors)}")
print(f"Med: {np.median(errors)}")

/tmp/ipykernel_3278/98128557.py:24: RuntimeWarning: divide by zero encountered in scalar divide
  h.coef[id] = (target_poly.coef[-it - 2] - g.coef[-1]*h_pow1.coef[-it - 2]) / (g.coef[-1] * (h_pow2.coef[-it - 2] - h_pow1.coef[-it - 2]))
/tmp/ipykernel_3278/98128557.py:24: RuntimeWarning: invalid value encountered in scalar subtract
  h.coef[id] = (target_poly.coef[-it - 2] - g.coef[-1]*h_pow1.coef[-it - 2]) / (g.coef[-1] * (h_pow2.coef[-it - 2] - h_pow1.coef[-it - 2]))
/tmp/ipykernel_3278/98128557.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  h.coef[0] = (target_poly.coef[-it - 3] - g.coef[-1]*h_pow1.coef[-it - 3] - g.coef[-2]*h.coef[-1]**(g_deg-1)) / (g.coef[-1] * (h_pow2.coef[-it - 3] - h_pow1.coef[-it - 3]))
/tmp/ipykernel_3278/98128557.py:31: RuntimeWarning: invalid value encountered in scalar subtract
  h.coef[0] = (target_poly.coef[-it - 3] - g.coef[-1]*h_pow1.coef[-it - 3] - g.coef[-2]*h.coef[-1]**(g_deg-1)) / (g.coef[-1] * (h_pow2.coef[-it - 3] - h_pow1.co

Max: 3.382673646212123e+39
Avg: 3.410015406987034e+36
Std: 1.073458469997168e+38
Med: 2.6670477382140234e-08
